<a href="https://colab.research.google.com/github/Muntasir2179/vector-database-learning/blob/main/VD_SQLite_Vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLite Database

There are some open source vector database available in the internet. One of them is `Chromadb`.

https://docs.trychroma.com/

In [1]:
import sqlite3

In [2]:
# create a connection to SQLite DB
conn = sqlite3.connect("sample.db")

In [3]:
# Create a cursor
'''
The cursor objects is going to help us execute all the SQL commands
'''
cursor = conn.cursor()

## Now let's create a table

Here we are going to create a table. It will be a `stocks` table. There will be two columns -

* stock_code
* stock_name

In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS stocks(
  stock_code INTEGER PROMARY KEY,
  stock_name TEXT NOT NULL
)
""")

In [10]:
# let's now insert some data
cursor.execute("INSERT INTO stocks (stock_name) VALUES (?)", ('TESLA',))
cursor.execute("INSERT INTO stocks (stock_name) VALUES (?)", ('Microsoft',))

In [12]:
# select records
cursor.execute("SELECT * FROM stocks")

In [13]:
rows = cursor.fetchall()
rows

[(None, 'TESLA'), (None, 'TESLA'), (None, 'Microsoft')]

In [14]:
# save the changes
conn.commit()

In [15]:
# it is a good practise to always close the database connection whenever we connect with some database
conn.close()

## Using SQLite as a vector storage

What is a vector?
> The vectors in machine learning signify input data, including bias and weight. In the same way, output from a machine-learning model (for example, a predicted class), can be put into vector format.

```python
# array of numbers -> numpy arrays
vector = [1.2, 2.5, 3.7, 7.5, 5.9]
```

🧮 NOTE: The information, in orther words the vectors must be stored in a bytes format.

In [33]:
import numpy as np

# creating a new connection to store vectors
conn = sqlite3.connect("sample_vectors.db")

# creting cursor to execute SQL commands
cursor = conn.cursor()

In [34]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS vectors (
  vector_id INTEGER PRIMARY KEY,
  vector BLOB NOT NULL
)
""")

In [35]:
# creating some vectors as numpy array
vector_tesla = np.array([1.4, 3.5, 2.2, 0.9])
vector_microsoft = np.array([2.8, 1.6, 3.8, 2.2])

In [36]:
# we have to convert our vector into bytes format before inderting into the database
vector_tesla.tobytes()

b'ffffff\xf6?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'

In [37]:
# we have to specify sqlite3 that it is stored in Binary format
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vector_tesla.tobytes()),))

In [38]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vector_microsoft.tobytes()),))

In [39]:
cursor.execute("SELECT * FROM vectors")
rows = cursor.fetchall()
rows

[(1,
  b'ffffff\xf6?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'),
 (2,
  b'ffffff\x06@\x9a\x99\x99\x99\x99\x99\xf9?ffffff\x0e@\x9a\x99\x99\x99\x99\x99\x01@')]

## Retriving vecotr from the database

Now we can see that the data has been converted into bytes and stored in the database. But, when we will try to retrive the data, we will not going to get the data in the format that we have inserted. We have to do some transformation to get the data/vector in actual format.

The process is called `Deserialization`.

In [41]:
rows[0][1]

b'ffffff\xf6?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'

In [44]:
# applying deserialization
vector = np.frombuffer(rows[0][1], dtype=np.float64)
vector

array([1.4, 3.5, 2.2, 0.9])

In [46]:
# retriving all the vectors
vectors = []
for row in rows:
  vectors.append(np.frombuffer(row[1], dtype=np.float64))
vectors

[array([1.4, 3.5, 2.2, 0.9]), array([2.8, 1.6, 3.8, 2.2])]